In [3]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\chatbot\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\chatbot\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\chatbot\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
from elastic_search.utilities.login import login
from utilities.es_search import search_es
from transformers import pipeline
import json

# Variables

In [7]:
# Get index name
with open('..\\elastic_search\\utilities\\index_config.json') as file:
    index_name = json.load(file)[0]['index_name']

# Connect to elastic search

In [8]:
es = login()

Connecting to elastic search...


C:\Users\chatbot\PycharmProjects\chatbot_project\venv\lib\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


Connection to elastic search successfully established


C:\Users\chatbot\PycharmProjects\chatbot_project\venv\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# Search in index

In [9]:

question_text = 'Was steht im Fokus des Studiums?'

es_answers = search_es(es_instance=es, index_name=index_name, question_text=question_text, number_of_results=10)

print(f'Question: {question_text}')
print(f'Query Duration: {es_answers["took"]} milliseconds')
print('Title, Relevance Score:')
[(hit['_source']['document_title'], hit['_score']) for hit in es_answers['hits']['hits']]


Question: Was steht im Fokus des Studiums?
Query Duration: 20 milliseconds
Title, Relevance Score:


C:\Users\chatbot\PycharmProjects\chatbot_project\venv\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[('Wirtschaftsinformatik-Application Management', 5.699464),
 ('Studienziele', 3.606553),
 ('Studienportal', 2.5081196),
 ('Studienvorbereitung Mathematik', 2.425592),
 ('Studieninhalte', 1.669601),
 ('Studienbeginn & Regelstudienzeit', 1.3914851),
 ('Eckdaten Wirtschaftsinformatik-Application Management', 0.92852104),
 ('Job & Karriere', 0.792441)]

# Define QA Pipeline


In [10]:
qas = pipeline(
    model='deutsche-telekom/bert-multi-english-german-squad2',
    task='question-answering'
    )

In [11]:
# Let the model get a answer

In [12]:
answer = qas(
    question=question_text,
    context=es_answers['hits']['hits'][0]['_source']['document_text']
    )
print(answer)

{'score': 0.07189861685037613, 'start': 262, 'end': 286, 'answer': 'aktuelle IT-Technologien'}


# Get the best answer according to Happy face' score from the 10 most relevant database entry's according to elastic search

In [25]:
best_answer = {'score': 0}

for hit in es_answers['hits']['hits']:
    answer = qas(
        question=question_text,
        context=hit['_source']['document_text']
        )
    if answer['score'] > best_answer['score']:
        best_answer = answer

print(best_answer)

{'score': 0.7899596095085144, 'start': 59, 'end': 75, 'answer': 'Theorie & Praxis'}


In [13]:
# Write the answer to a JSON file

In [14]:
answer_dict = dict([
    ("question", question_text),
    ("answer", best_answer["answer"]),
    ("score", best_answer['score']),
    ('link', es_answers['hits']['hits'][0]['_source']['link'])
    ])

with open('answer.json', 'w') as file:
    json.dump(answer_dict, file, ensure_ascii=False)
